In [21]:
import pandas as pd
import numpy as np
import logging
from scripts.models.agent import Agent
from scripts.models.parameters import Parameters
import scripts.main as main
import plotly.graph_objects as go
import plotly
import random
from plotly.subplots import make_subplots  
import os
from datetime import datetime
from collections import defaultdict

In [22]:
generate = True
num_agents = 10

productivities = [100] * num_agents
num_steps = 30

pars = Parameters()
pars.n = num_agents

probability = 0.5

def is_neib(i, j):
    rand = random.random()

    if abs(i - j) == 1:
        return 1

    if rand < probability:
        return 1
    
    return 0
    

Adj = np.matrix([[is_neib(i, j) for i in range(num_agents)] for j in range(num_agents)])
print(Adj)
W = Adj / 2
# Adj = np.array([
#         [0, 0, 1, 1, 0],
#         [0, 0, 0, 1, 1],
#         [1, 0, 0, 0, 1],
#         [1, 1, 0, 0, 0],
#         [0, 1, 1, 0, 0]
#     ])
# W = Adj / 2
pars.b = W

pars.neib_add = 5
pars.add_neib_val = 0.5

# L_sub =  max([sum([W[i, j] * W[i, j] for j in range(num_agents)])/productivities[i]**4 for i in range(num_agents)])
# L = np.sqrt((num_agents + 1) * L_sub)

# mu = min([sum(W[i].A1)/productivities[i]**2 for i in range(num_agents)])

pr = max([productivities[agent] for agent in range(num_agents)])
L = (0.5*0.5 * 2) * (num_agents + 1) / pr
mu = 0.9
h = 0.002
eta = 0.8
alpha = [0.0001, 0.003, 0.011][1]
gamma = [[0.007, 0.009, 0.011][0]]

# h = 0.2
# eta = 0
# alpha = [0.07, 0.09, 0.11][2]
# gamma = [[0.07, 0.09, 0.11, 0.4][3]]

pars.params_dict = {
    "L": L,
    "mu": mu,
    "h": h,
    "eta": eta,
    "gamma": gamma,
    "alpha": alpha
}

print(pars.params_dict)


# gamma.append((1 - alpha) * gamma[0] + alpha * (mu - eta))
# H = h - h * h * L / 2
# H - alpha * alpha / (2 * gamma[1]) #should be more than zero

[[0 1 0 1 0 1 1 0 0 1]
 [1 1 1 1 0 0 1 1 0 1]
 [0 1 0 1 0 1 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 0]
 [0 1 0 1 0 1 0 0 1 1]
 [1 0 0 0 1 0 1 1 0 0]
 [0 1 1 1 1 1 1 1 1 1]
 [0 1 0 0 1 1 1 1 1 0]
 [1 0 1 0 0 1 1 1 1 1]
 [1 1 0 1 1 0 1 1 1 0]]
{'L': 0.055, 'mu': 0.9, 'h': 0.002, 'eta': 0.8, 'gamma': [0.007], 'alpha': 0.003}


In [23]:
exp = 1
iter_past = 0

def add_error(df, column_name, num):
    df["Mean"] = df.mean(axis=1)
    for i in range(num):
        df[f"{column_name}_{i}_sqr"] = np.power(df[f"{column_name}_{i}"] - df[f"Mean"], 2)
    df["Error"] = np.sqrt(sum([df[f"{column_name}_{i}_sqr"] for i in range(num)])/num)
    return df

def plot(seq_df, num_agents, alg):
    """ Plot queue length dynamics """
    global k, exp
    
    layout = go.Layout(plot_bgcolor='rgba(0,0,0,0)')
    fig = make_subplots(rows=2, cols=1)

    for i in range(num_agents):
        sequence = seq_df[f"Agent_{i}"]
        fig.add_trace(go.Scatter(y=sequence,          
                                 x=list(range(len(sequence))),
                                             name=f"Agent {i} {alg}",
                                             mode="lines"),
                     row=1, col=1)

    fig.update_layout(
        xaxis_title="t",
        yaxis_title="Queue length",
        font = dict(size = 20),
        height=600,
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.show()

    fig.write_html(f"./trash/trash_{k}_{exp}_{alg}_seq.html");
    
def run(is_evaluate, algorithm, num_agents, num_steps, pars, generate, productivities, ret_alg=False):
    global k, exp
    
    alg = None
    if is_evaluate:
        print(algorithm)
        alg = main.run_load_balancing(algorithm, num_steps, pars, generate, productivities=productivities)
    
        print(algorithm)
        alg_seq = alg.sequence
        seq_df = pd.DataFrame(alg_seq, columns=[f"Agent_{ind}" for ind in range(num_agents)])
        
        plot(seq_df, num_agents, alg=algorithm)
        
        seq_df = add_error(seq_df, "Agent", num_agents);
        seq_df.to_csv(f"./trash/seq/{exp}_{k}_{algorithm}_seq.html")
    else:
        seq_df = pd.read_csv(f"./trash/seq/{exp}_{k}_{algorithm}_seq.html")
        print(f"Added seq_df")
    
    if ret_alg:
        return alg, seq_df
    return None, seq_df

In [24]:
itr = 1
start = -1
iter_past = 0

for k in range(iter_past, itr):
    # Run LVP and extract sequnce
    do = k > start
    alg_lvp, lvp_seq = run(do, main.LVP, num_agents, num_steps, pars, True, productivities, ret_alg=True) 

    # Run ALVP
    do = k > start
    alg_alvp, alvp_seq = run(do, main.ALVP, num_agents, num_steps, pars, False, productivities, ret_alg=True) 

    # Run RR
    do = k > start
    alg_rr, rr_seq = run(do, main.ROUND_ROBIN, num_agents, num_steps, pars, False, productivities, ret_alg=True) 
    
    # Plot error plot
    layout = go.Layout(plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(layout=layout)

    fig.add_trace(go.Scatter(y=lvp_seq["Error"],
                                         x=list(range(len(lvp_seq))),
                                         name=f"Error LVP",
                                         mode="lines",
                            line={'dash': 'dot', 'color': 'red'}))

    fig.add_trace(go.Scatter(y=alvp_seq["Error"],
                                 x=list(range(len(alvp_seq))),
                                 name=f"Error ALVP",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'green'}))


    fig.add_trace(go.Scatter(y=rr_seq["Error"],
                                 x=list(range(len(rr_seq))),
                                 name=f"Error RR",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'blue'}))


    fig.update_layout(
        xaxis_title="t",
        yaxis_title="Error",
        legend = dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.9
        ),
        font = dict(size = 20),
        height=800
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.show()

    fig.write_html(f"./trash/trash_{k}_{exp}_error.html");        
    
print("Finish")
iter_past = k

LVP
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
Step 28 is completed
Step 29 is completed
LVP


ALVP
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
Step 28 is completed
Step 29 is completed
ALVP


ROUND_ROBIN
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
Step 28 is completed
Step 29 is completed
ROUND_ROBIN


Finish
